In [1]:
# Real v. Synthesized Image
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [20]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
import torchvision
from torchvision import transforms
from PIL import Image

import sys
sys.path.append('/content/gdrive/Shareddrives/Advanced CV - Final Project/GradCAM/')

from InceptionV3 import*

In [15]:
preprocess = transforms.Compose([
    #resizes the image to the given size
    #argument: desired square output size
    #scales the image
    transforms.Resize(299),
    #center crop: crops the image at the center, argument is the desired h,w
    #so here only one argument, output will be 299 x 299
    transforms.CenterCrop(299),
    #converts an image to a tensor
    transforms.ToTensor(),
    #normalize a tensor image with mean and std deviation provided
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [21]:
def gradCAM(img_input, model, intensity=0.5, res=250):
  img = image.load_img(img_input, target_size=(299, 299))
  test_image = Image.open(img_input)
  tensor_image = preprocess(test_image)

  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)

  preds = model(tensor_image)
  print(preds)
  pred = torch.argmax(preds, dim=1)
  # print(decode_predictions(preds)[0][0][1]) # prints the class of image

  with tf.GradientTape() as tape:
    last_conv_layer = model.get_layer('conv2d_93')
    iterate = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output])
    model_out, last_conv_layer = iterate(x)
    class_out = model_out[:, np.argmax(model_out[0])]
    grads = tape.gradient(class_out, last_conv_layer)
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    
  heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)
  heatmap = np.maximum(heatmap, 0)
  heatmap /= np.max(heatmap)
  heatmap = heatmap.reshape((8, 8))

  img = cv2.imread(img_input)

  heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

  heatmap = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

  img = heatmap * intensity + img

  cv2_imshow(cv2.resize(cv2.imread(orig), (res, res)))
  cv2_imshow(cv2.resize(img, (res, res)))


In [4]:
# paths to saved model dict.
path_to_model = '/content/gdrive/Shareddrives/Advanced CV - Final Project/GradCAM/checkpoints/epoch-9.pt'

In [ ]:
# load model dictionary
device = torch.device('cpu')
model = Inception3(num_classes = 2)
model.load_state_dict(torch.load(path_to_model, map_location=device))
model.eval()

In [6]:
# paths to images
path_to_image = '/content/gdrive/Shareddrives/Advanced CV - Final Project/GradCAM/seed0608.jpeg'

In [ ]:
gradCAM(path_to_image, model)